In [2]:
pip install requests beautifulsoup4 requests-html


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession
import requests
from bs4 import BeautifulSoup

In [4]:
def get_and_parse(url):
  response = requests.get(url)

# Parse the HTML content with Beautiful Soup
  soup = BeautifulSoup(response.content, 'html.parser')

# Remove unwanted elements (script, style, comments, etc.)
  for element in soup(['script', 'style', 'comment']):
      element.extract()

# Extract the visible text
  text = soup.get_text(separator=' ')
  return text

In [5]:
url="https://en.wikipedia.org/wiki/Artificial_neural_network"
result=get_and_parse(url)
#print(result)

In [6]:
def parse_elements(url,element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    return elements

In [7]:
url = "https://en.wikipedia.org/wiki/Artificial_neural_network"
result=parse_elements(url, 'p')
#print(result)
Re=[]

for i in result:
  Re.append(i.text)

Re_T=np.reshape(Re,(len(Re),1))
#print(Re_T)

In [8]:
def topic_detection(sentence):
    DataScience_words = ('data science', 'regression', 'predict', 'predictive', 'classify','classification')
    AI_words = ('AI', 'Artificial intelligence','machine learning', 'ml', 'ML', 'learn','neurons','training', 'NLP','deep learning','neural network','ANN', 'Fuzzy logic','heuristic', 'Genetic algorithm', 'GA', 'particle swarm optimization')
    DataWords = ('analysis', 'algorithms','data','data structure','data-structure','big data')


    DataScience = any(sentence.count(i) > 0 for i in DataScience_words)
    AI = any(sentence.count(i) > 0 for i in AI_words)
    data = any(sentence.count(i) > 0 for i in DataWords)

    topics = []
    if DataScience == True:
        topics.append("Data Science")
    if AI == True:
        topics.append("AI")
    if data == True:
        topics.append("data")

    return topics

In [9]:
#requirements: print out a topic analysis after every paragraph prints out
#dictionary that has two lists, one for the paragraphs, and one for topics covered by those paragraphs
#print(paragraphs)
p_dictionary = {}
p_list = []
topic_list = []
for p in range(0, len(result)):
    if len(result[p].text) > 100:
      #print((paragraphs[p].text))
      #print(f"TALKS ABOUT: {topic_detection(paragraphs[p].text)}")
      p_list.append(result[p].text)
      topic_list.append(topic_detection(result[p].text))
p_dictionary['Paragraphs'] = p_list
p_dictionary['Topics'] = topic_list

In [10]:
medium_df = pd.DataFrame(p_dictionary)
medium_df.head(10)


,Paragraphs,Topics
0,"Artificial neural networks (ANNs, also shorten...",[AI]
1,An ANN is based on a collection of connected u...,[AI]
2,"Typically, neurons are aggregated into layers....",[AI]
3,Neural networks learn (or are trained) by proc...,"[Data Science, AI, data]"
4,"Such systems ""learn"" to perform tasks by consi...",[AI]
5,The simplest kind of feedforward neural networ...,"[Data Science, AI]"
6,Wilhelm Lenz and Ernst Ising created and analy...,[AI]
7,Warren McCulloch and Walter Pitts[11] (1943) a...,[AI]
8,Some say that research stagnated following Min...,[AI]
9,The first deep learning MLP was published by A...,"[Data Science, AI]"


In [11]:
medium_df['Character Count'] = medium_df['Paragraphs'].apply(lambda x: len(x))
medium_df.head(10)

,Paragraphs,Topics,Character Count
0,"Artificial neural networks (ANNs, also shorten...",[AI],282
1,An ANN is based on a collection of connected u...,[AI],778
2,"Typically, neurons are aggregated into layers....",[AI],548
3,Neural networks learn (or are trained) by proc...,"[Data Science, AI, data]",832
4,"Such systems ""learn"" to perform tasks by consi...",[AI],571
5,The simplest kind of feedforward neural networ...,"[Data Science, AI]",691
6,Wilhelm Lenz and Ernst Ising created and analy...,[AI],329
7,Warren McCulloch and Walter Pitts[11] (1943) a...,[AI],585
8,Some say that research stagnated following Min...,[AI],354
9,The first deep learning MLP was published by A...,"[Data Science, AI]",471


In [12]:
medium_df['Topic Count'] = medium_df['Topics'].apply(len)
medium_df.head()

,Paragraphs,Topics,Character Count,Topic Count
0,"Artificial neural networks (ANNs, also shorten...",[AI],282,1
1,An ANN is based on a collection of connected u...,[AI],778,1
2,"Typically, neurons are aggregated into layers....",[AI],548,1
3,Neural networks learn (or are trained) by proc...,"[Data Science, AI, data]",832,3
4,"Such systems ""learn"" to perform tasks by consi...",[AI],571,1
